In [1]:
import pandas as pd
import numpy as np
import random 
import os
from IPython.display import clear_output

In [2]:
df = pd.read_excel("単語データ.xlsx",header=None)
df.columns = ("number","English","Japanese")
df = df.set_index("number")
wrong = df["Japanese"].unique()
wrong_list = pd.DataFrame(wrong)
df.head()

,English,Japanese
number,,
1,a number of ~,いくつもの~;かなり多くの~
2,a piece of ~,1つ[1個;1本;1枚;...]の~
3,a couple of ~,"12つの~;2人の~ 22,3の~"
4,a great[good] deal (of ~),かなりたくさん(の~)
5,plenty of ~,たくさんの~


In [3]:
df.describe()

,English,Japanese
count,100,100
unique,100,100
top,a number of ~,いくつもの~;かなり多くの~
freq,1,1


In [4]:
def testmake(n,start,stop) -> pd.DataFrame:
    #テスト範囲の取得
    test_range = df.iloc[start-1:stop]
    test_sampled = test_range.sample(n = n,replace = False)
    
    #テストの選択肢の作成
    for i in range(4):
        test_sampled["test_{}".format(i+1)] = None
    test_sampled["answer"] = None
    test = test_sampled.iloc[0:0]
  
    #選択肢に正解と問題を入れる    
    for i in range(0,len(test_sampled)):
        test_question = test_sampled.iloc[[i],:].copy()
        answer = random.randint(1,4)
        test_question["test_{}".format(answer)] = test_question["Japanese"]

        test_wrong = wrong_list.sample(n = 4)
        while pd.concat([test_wrong,test_question["Japanese"]]).nunique().sum() <= 4:
            test_wrong = wrong_list.sample(n = 4,replace=False)
        
        for wro in range(1,5):
            if test_question["test_{}".format(wro)].iloc[[0]].isnull().bool() == True:
                test_question["test_{}".format(wro)].iloc[[0]] = test_wrong.iloc[wro-1]
                
        test_question["answer"] = answer
        
        test = pd.concat([test,test_question])
        
    del test["Japanese"]
    return test

def test_play(test_df):
    cor = 0
    test_df["is_correct"] = None
    for i in range(len(test_df)):
        print(i + 1,"問目","-------------------")
        finish = True
        while finish == True:
            display(test_df.iloc[i,0:5])
            answer = int(input("答えを入力してください。もし表示されなかったら0"))
            if answer == test_df.iloc[i,5]:
                test_df.iloc[i,6] = 1
                cor += 1
                finish = False
            elif answer == 0:
                finish = True
            else:
                test_df.iloc[i,6] = 0
                finish = False
        
        if (i + 1) % 5 == 0 and i != 0:
            clear_output()

        

In [8]:
test_test = testmake(n = 15,start = 1,stop = 50)
print(len(test_test))
test_test

15


,English,test_1,test_2,test_3,test_4,answer
number,,,,,,
16,be different from ~,~に集中する,~に参加する,大人になる;(事態などが)生じる,~とは違っている,4
23,be based on[upon] ~,~を置いていく;~を置き忘れる,これに反して;他方では,大人になる;(事態などが)生じる,~に基づいている,4
29,think of ~,1働き続ける 2~を製作する;~に取り組む;~に影響を与える,~に満足している,1~のことを考える;~しようかなと思う 2~を思いつく 3~を思い出す,~を捜[探]す,3
7,close to ~,~のすぐ近くに;ほぼ~,初めて,~に満足している,~を立[建]てる;~を創設する;~を始める,1
13,all the time,いつも;常に,~どころではない;~からほど遠い,~に責任がある,1~を破壊する;~を取り壊す;~を分解する 2故障する;取り乱す;肉体的[精神的]に参る,1
48,participate in ~,~に参加する,ところで,1(get to +(代)名詞で)~に到着する 2(get to do で)~するようになる...,1すぐに 2同時に,1
30,refer to ~,1一般に 2(名詞の後に置いて)一般の,~に参加する,1~のことを考える;~しようかなと思う 2~を思いつく 3~を思い出す,1~に言及する 2~を参照する;~に問い合わせる,4
41,work on (~),~を立[建]てる;~を創設する;~を始める,1働き続ける 2~を製作する;~に取り組む;~に影響を与える,~を指摘する,~のすぐ近くに;ほぼ~,2
34,deal with ~,~を扱う;~を処理する,1~を育てる 2(問題・話題など)を持ち出す,~など,~を求める,1


In [9]:
test_test = testmake(n = 11,start = 1,stop = 50)
test_play(test_df=test_test)

1 問目 -------------------


English          far from ~
test_1     ~どころではない;~からほど遠い
test_2       いくつもの~;かなり多くの~
test_3               ~を指摘する
test_4          ~のすぐ近くに;ほぼ~
Name: 12, dtype: object

ValueError: invalid literal for int() with base 10: ''

In [129]:
test_test

,English,test_1,test_2,test_3,test_4,answer,is_correct
number,,,,,,,
24,be familiar to ~,~を尊敬する,~を確信している,~する傾向がある;~したい気がする,(人)によく知られている,4,1
28,as a matter of course,それどころか;それに反して,当然[もちろん]のこととして,AをBと間違える,突き出る;目立つ,2,0
5,be bound to do,きっと~する,現在は;目下,とにかく;いずれにしても,お互い,1,1
20,on one's part,~しがちである,~の側での;~のほうでは,実際は;実を言うと,~を心配している,2,0
4,be bound for ~,~から始まる;~から続いている,~にある,~行きである,AをBだと(誤って)思う;AをBと間違える,3,0
29,as a matter of fact,(物事)をよく知っている,全然~ではない;~どころではない,実際は;実を言うと,~について言えば;~に関する限り,3,0
17,keep pace with ~,(人)によく知られている,~を確信している,どんどん;延々と,~に遅れずについていく,4,0
14,not to mention ~,~としては;~に関する限りは,~は言うまでもなく,~にある,(物事)をよく知っている,2,0
12,with all ~,~にもかかわらず,全然~ではない;~どころではない,要するに;つまり,~としては;~に関する限りは,1,1


In [84]:
correct_list = pd.DataFrame(test_test.loc[test_test["is_correct"] == 1].index)
miss_list = pd.DataFrame(test_test.loc[test_test["is_correct"] == 0].index)

In [131]:
correct_list.T.values.tolist()

[[80, 97, 88]]